In [ ]:
import vectorManipulation as VM
import myServices as ms
import geopandas as gpd
from shapely.geometry import MultiPolygon, Polygon
from shapely.ops import unary_union
import utm
    

In [15]:
shpPath = '/Users/abdielfer/RNCanWork/CrossCanFloodSuscMap/HRDTMByAOI/BC_Bulton_2018_ok/BC_Bulton_Subset.shp'
shp = VM.importSHPFIles(shpPath)
# shp.print_headAttributeTable()
gpdObject = shp.getGPDObject()
print(shp.getCRS())

epsg:4617


In [ ]:
gpdObject.head(1)

In [16]:
# print(gpdObject['geometry'])

if gpdObject.loc[0,'geometry'].geom_type == 'MultiPolygon':
#     geomed = gpdObject.loc[0,'geometry'].geoms
#     print('geomed.....:', type(geomed),len(geomed))
#     [print(g) for g in geomed]
    explodedMultipoligon= gpdObject.explode(index_parts=True)
    newGeoDf = explodedMultipoligon.dissolve(by=sum)
    print(type(newGeoDf))
else:
    print(gpdObject.loc[0,'geometry'].area)

<class 'geopandas.geodataframe.GeoDataFrame'>


In [17]:
from pyproj import CRS

def findtheutm(aGeometry):
    '''
    A function to find a coordinates UTM zone
    '''   
    x, y, parallell, latband = utm.from_latlon(aGeometry.centroid.y, aGeometry.centroid.x)
    if latband in 'CDEFGHJKLM': #https://www.lantmateriet.se/contentassets/379fe00e09d74fa68550f4154350b047/utm-zoner.gif
        ns = 'S'
    else:
        ns = 'N'
    crs = "+proj=utm +zone={0} +{1}".format(parallell, ns) #https://gis.stackexchange.com/questions/365584/convert-utm-zone-into-epsg-code
    crs = CRS.from_string(crs)
    _, code = crs.to_authority()
    return int(code)
    
def computeGeometryArea(gpdFrame: gpd.geodataframe.GeoDataFrame):
    '''
        You must extrac de polygon from the GeoDataFrame befor passing to the function.
    '''
    epsg = findtheutm(gpdFrame.geometry.iloc[0])
    gpdFrame['Area_m2'] = gpdFrame.to_crs(epsg).area
    return gpdFrame

# newDfWithAreaOK = computeGeometryArea(newGeoDf)

In [19]:
epsg = findtheutm(newGeoDf.geometry.iloc[0])
print(epsg)
reporjGoeDF= newGeoDf.to_crs(epsg)
omega = unary_union([
    Polygon(g)for g in reporjGoeDF['geometry'] 
])
print(type(omega), omega.area)

32611
<class 'shapely.geometry.multipolygon.MultiPolygon'> 511426.07802608906


In [ ]:
newDfWithAreaOK

In [ ]:
if 'MULTIPOLYGON' in gpdObject.loc[0,'geometry'].geoms:
    print('Yeaaa..')
    

In [ ]:

print(VM.computePolygonArea(gpdObject.loc[0,'geometry']))